<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-07-20 08:39:17--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.2’

SO_vectors_200.bin? 100%[===================>]   1.35G  3.63MB/s    in 9m 22s  

2023-07-20 08:48:42 (2.47 MB/s) - ‘SO_vectors_200.bin?download=1.2’ saved [1453905423/1453905423]



In [ ]:
from gensim.models import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1.2", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

In [ ]:
wv_embeddings['dog']

array([ 0.6851772 , -1.2778991 , -0.41913974,  1.3623164 , -3.1675398 ,
        0.09950767,  0.6402681 , -1.1245339 , -0.6699619 , -0.6998852 ,
        0.4936771 , -0.40500194, -3.0706816 , -2.2809966 ,  0.85798043,
        2.7093108 ,  0.3492745 , -0.03494101, -0.22330493,  1.2290467 ,
        1.7755157 , -3.158358  , -0.6515983 ,  0.7224096 ,  2.3193083 ,
       -1.7969862 ,  0.40903398, -2.744604  , -1.7179952 , -0.914309  ,
       -0.75887376, -0.35140672, -0.5182776 , -1.9097351 , -0.8300773 ,
        0.02147918,  1.1783471 ,  0.03169126, -0.3069023 ,  1.6666299 ,
        0.6711357 , -2.1706133 , -0.11800487,  0.22336982, -1.2075394 ,
       -0.86297905, -0.63865614,  1.1733794 ,  0.10022762,  0.7017279 ,
        2.7290728 , -0.4640484 , -2.1719306 , -0.3562852 , -1.8449957 ,
        0.10270727,  1.1125596 , -0.8364318 ,  1.9513408 , -0.97937447,
        1.2650859 ,  0.06809282,  0.6477318 , -0.52431005, -0.6103959 ,
       -2.979829  , -0.7889965 , -0.11004248,  1.7603841 , -1.05

In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_similar
print(wv_embeddings.most_similar('dog', topn=5))

[('animal', 0.8564180135726929), ('dogs', 0.7880866527557373), ('mammal', 0.7623804211616516), ('cats', 0.7621253728866577), ('animals', 0.760793924331665)]


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
# import numpy as np
# import re
# # you can use your tokenizer
# # for example, from nltk.tokenize import WordPunctTokenizer
# class MyTokenizer:
#     def __init__(self):
#         pass
#     def tokenize(self, text):
#         return re.findall('\w+', text)
# tokenizer = MyTokenizer()

In [ ]:
# tokenizer.tokenize("The price\t of burger \nin BurgerKing is Rs.36.\n")

In [ ]:
from nltk.tokenize import WordPunctTokenizer
my_tokenizer = WordPunctTokenizer()

In [ ]:
my_tokenizer.tokenize("The price\t of burger \nin BurgerKing is Rs.36.\n")

['The',
 'price',
 'of',
 'burger',
 'in',
 'BurgerKing',
 'is',
 'Rs',
 '.',
 '36',
 '.']

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    ans = []
    stop_words = set(stopwords.words('english'))
    lem = WordNetLemmatizer()
    tk = tokenizer.tokenize(question.lower())
    words = [lem.lemmatize(w) for w in tk if w not in stop_words]
    count = 0
    for word in tk:
        if word in embeddings:
            count += 1
            ans.append(embeddings[word])
    if count == 0:
        return np.zeros(200, dtype='float32')
    else:
        return np.mean(ans, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
sentences = 'I love neural networks'
vector = question_to_vec(sentences, wv_embeddings, my_tokenizer, dim=200)

In [ ]:
vector

array([-1.0142275 , -1.6891261 , -1.2854122 , -1.3710302 ,  0.15916634,
        0.06706961,  1.1188413 , -0.07051762,  0.5386369 , -0.37139115,
        0.28913018,  1.03125   , -2.865466  , -1.3192873 ,  2.2098296 ,
        1.1678672 ,  1.0841951 ,  1.8467226 , -0.98733807,  0.8720946 ,
       -0.17219625,  0.73839015, -0.7827826 , -0.8827869 , -0.6152611 ,
       -0.15872358, -0.6170798 ,  0.36668167,  2.0137532 ,  0.2627472 ,
       -1.4950123 , -1.386715  , -1.2049937 , -1.016617  ,  2.7621174 ,
        1.7255713 , -0.19038178, -0.5899447 , -0.1141155 , -1.8890352 ,
       -1.7883505 , -0.20420384, -0.7062034 ,  2.489034  , -0.15452717,
        1.1255013 ,  0.2610835 ,  1.3153247 , -0.57667524, -0.3327141 ,
        1.6764902 ,  0.91229993, -0.69880134,  2.3752553 , -1.7634224 ,
        3.1378052 ,  1.2243204 , -1.0554415 ,  0.7897571 ,  0.04404024,
        1.1623045 , -0.4478023 , -0.9149816 ,  0.01501505, -0.85523087,
       -1.0183454 , -0.7162116 ,  1.0727204 ,  0.8881876 , -0.29

In [ ]:
round(vector[2],2)

-1.29

In [ ]:
sentences = 'dslskk dfjddhj'
vector = question_to_vec(sentences, wv_embeddings, my_tokenizer, dim=200)
vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

Вычислим метрику *DCG@K* для *K = 10*:
- [K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = 0,3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    value = []
    for num in dup_ranks:
        value.append(int(num <= k))
    hits_value = np.mean(value)
    return hits_value

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    value = []
    for num in dup_ranks:
      value.append(int(num <= k) / np.log2(1 + num))
    dcg_value = np.mean(value)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: __MACOSX/data/._train.tsv  
  inflating: data/validation.tsv     
  inflating: __MACOSX/data/._validation.tsv  


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54517
2 52818
3 54464
4 53724
5 52297


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    new_c = []
    vector_q = question_to_vec(question, embeddings, tokenizer, dim=200)
    vector_q = vector_q.reshape(1, -1)
    for j in range(len(candidates)):
        vector_c = question_to_vec(candidates[j], embeddings, tokenizer, dim=200)
        vector_c = vector_c.reshape(1, -1)
        dist = cosine_similarity(vector_c, vector_q)
        dist = np.reshape(dist, -1)
        dist = dist.tolist()
        new_c.append([dist[0], j, candidates[j]])
    new_c = sorted(new_c, reverse=True)
    for i in range(len(new_c)):
        new_c[i] = tuple(new_c[i][1:])
    return new_c

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, my_tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# # должно вывести
# results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (*, 'select2 not displaying search results'), #скрыт
#             (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    line = line.split('	')
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, my_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x7f2bdb3de7a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/nltk/data.py", line 1160, in __del__
    if not self.closed:
  File "/usr/local/lib/python3.10/dist-packages/nltk/data.py", line 1180, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.494 | Hits@   5: 0.578
DCG@  10: 0.516 | Hits@  10: 0.644
DCG@ 100: 0.563 | Hits@ 100: 0.875
DCG@ 500: 0.575 | Hits@ 500: 0.973
DCG@1000: 0.578 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
train_data

In [ ]:
words = deepcopy(train_data)
stop_words = set(stopwords.words('english'))
lem = WordNetLemmatizer()

In [ ]:
tk =[]
for i in range(len(words)):
    words[i] = words[i].split('\t')
    words[i] = (' ').join(words[i])
    tk.append(my_tokenizer.tokenize(words[i].lower()))

In [ ]:
for i in range(len(tk)):
    tk[i] = [lem.lemmatize(w) for w in tk[i] if w not in stop_words]

In [ ]:
len(tk)

1000000

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    line = line.split('	')
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, my_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.017 | Hits@   1: 0.017
DCG@   5: 0.029 | Hits@   5: 0.039
DCG@  10: 0.031 | Hits@  10: 0.046
DCG@ 100: 0.052 | Hits@ 100: 0.161
DCG@ 500: 0.074 | Hits@ 500: 0.335
DCG@1000: 0.143 | Hits@1000: 1.000
